In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import PIL
import matplotlib.pyplot as plt

In [2]:
class EmotionDataset:
    def __init__(self, parent_folder: str, mode: str = 'train', shuffle=True):
        self.parent_folder = parent_folder
        self.data = []
        self.mode = mode
        self.labels={'angry':0, 'disgusted':1, 'scared':2, 'happy':3, 'normal':4, 'sad':5, 'surprised':6}
    
        subset = self.mode
        subset_path = os.path.join(parent_folder, subset)
        for emotion in os.listdir(subset_path):
            emotion_path = os.path.join(subset_path, emotion)
            if os.path.isdir(emotion_path):
                for file_name in os.listdir(emotion_path):
                    file_path = os.path.join(emotion_path, file_name)
                    if os.path.isfile(file_path):
                        image = PIL.Image.open(file_path)
                        self.data.append((image, self.labels[emotion.lower()]))
        
        if shuffle:
            np.random.shuffle(self.data)

    def __getitem__(self, index: int):
        image, label = self.data[index]
        fd, _ = hog(
                        image,
                        orientations=8,
                        pixels_per_cell=(32, 32),
                        cells_per_block=(8, 8),
                        visualize=True,
                        channel_axis=-1,
                    )
        return  fd, label

    def __len__(self) -> int:
        return len(self.data)

In [3]:
train_dataset = EmotionDataset('Master folder')
valid_dataset = EmotionDataset('Master folder', 'valid')

In [4]:
def dataset_creation(dataset: EmotionDataset):
    X, y = [], []
    for image, label in dataset:
        X.append(image)
        y.append(label)
    return np.array(X), np.array(y)

In [5]:
train_x, train_y = dataset_creation(train_dataset)
val_X, val_y = dataset_creation(valid_dataset)

In [6]:
clf = svm.SVC()
clf.fit(train_x, train_y)
print(accuracy_score(val_y, clf.predict(val_X)))

0.28402366863905326
